In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

 test for https://github.com/frankwswang/Quiqbox.jl/blob/main/test/unit-tests/SubModule/Moldens/Test_H2_STO-3G-H2_RHF.molden

In [2]:
symbols = ['H','H']
geometry = np.array([[-0.70014,0.0,0.0],[0.70014,0.0,0.0]],requires_grad=False)
l_tuple_array = [(0,0,0),(0,0,0)]
exponent_array = np.array([[3.42525,0.62391,0.16886],[3.42525,0.62391,0.16886]],requires_grad=True)
contract_coeff_array = np.array([[0.15433,0.53533,0.44463],[0.15433,0.53533,0.44463]],requires_grad=True)

H2 = qchem.Molecule(symbols,geometry,charge=0,mult=1,basis_name='sto-3g',
                        l=l_tuple_array,alpha=exponent_array,coeff=contract_coeff_array)
print(H2.charge)
print(H2.mult)
print(H2.l)
print(H2.alpha)
print(H2.coeff)
args = [exponent_array,contract_coeff_array]
# SCF optimization of basis params using Pennylane
qchem.hf_energy(H2)(*args)

0
1
[(0, 0, 0), (0, 0, 0)]
[[3.42525 0.62391 0.16886]
 [3.42525 0.62391 0.16886]]
[[0.15433 0.53533 0.44463]
 [0.15433 0.53533 0.44463]]


array([-1.11670544])

In [3]:
qchem.electron_integrals(H2)(*args)

(array([0.71414289]),
 array([[-1.25270909e+00, -2.13939977e-13],
        [-2.13828955e-13, -4.75696482e-01]]),
 array([[[[6.74569846e-01, 2.46663800e-13],
          [2.46774823e-13, 1.81266827e-01]],
 
         [[2.46774823e-13, 1.81266827e-01],
          [6.63540037e-01, 2.27928787e-13]]],
 
 
        [[[2.46608289e-13, 6.63540037e-01],
          [1.81266827e-01, 2.28039809e-13]],
 
         [[1.81266827e-01, 2.27595720e-13],
          [2.27928787e-13, 6.97467874e-01]]]]))

In [185]:
file_path = 'structures/Test_H2_STO-3G-H2_RHF.molden'
molden_file_list = []
with open(file_path,'r') as f:
    molden_file_list.append(f.readlines())

molden_file = molden_file_list[0]
for i,line in enumerate(molden_file):
    if 'Atoms' in line:
        n_atom = 0
        future_line = 'start'
        atom_list = []
        atom_coords = []
        while '\n' != future_line:
            x = molden_file[i+n_atom+1][:-1].split('    ',8)
            x = list(filter(None,x))
            atom_list.append(x[:3])
            atom_coords.append(np.array(x[3:],dtype=np.float32))
            n_atom+=1
            future_line = molden_file[i+n_atom+1]
            #print(np.float32(x[3]))
    if 'GTO' in line:
        ind = 0
        param_list = []
        for j in range(n_atom):
            x = molden_file[i+ind+1][:-1].split(' ',2)
            x = list(filter(None,x))
            y = molden_file[i+ind+2][:-1].split('  ',4)
            y = list(filter(None,y))
            ind+=3
            params = np.zeros((int(y[1]),2))
            l = 0
            for k in range(ind, ind+int(y[1])):
                x = molden_file[i+k][:-1].split('     ',2)
                x = list(filter(None,x))
                x = np.array(x,dtype=np.float32)
                params[l,:] = x
                ind+=1
                l+=1
            param_list.append(params)

In [187]:
print(atom_list)
print(param_list)
print(atom_coords)

[['H', '1', '1'], ['H', '2', '1']]
[tensor([[3.42525005, 0.15433   ],
        [0.62391001, 0.53533   ],
        [0.16886   , 0.44463   ]], requires_grad=True), tensor([[3.42525005, 0.15433   ],
        [0.62391001, 0.53533   ],
        [0.16886   , 0.44463   ]], requires_grad=True)]
[tensor([-0.70014,  0.     ,  0.     ], dtype=float32, requires_grad=True), tensor([0.70014, 0.     , 0.     ], dtype=float32, requires_grad=True)]
